Второй для анализа я выбрал базовую модель knn, как она работает ищет

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import accuracy_score, f1_score
import time
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
split_idx = int(0.8 * len(train))
test = train.iloc[split_idx:]
train = train.iloc[:split_idx]

X_train = train.drop('Fertilizer Name', axis=1)
y_train = train['Fertilizer Name']

X_test = test.drop('Fertilizer Name', axis=1)
y_test = test['Fertilizer Name']


from sklearn.preprocessing import OneHotEncoder
cat_cols = ['Soil Type', 'Crop Type']
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[cat_cols]).toarray()
feature_names = encoder.get_feature_names_out(cat_cols)
X_test_encoded = encoder.transform(X_test[cat_cols]).toarray()

from sklearn.preprocessing import StandardScaler
num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

X_train_processed = np.hstack([X_train_scaled, X_train_encoded])
X_test_processed = np.hstack([X_test_scaled, X_test_encoded])

results = {}

In [3]:
print("NaN в X_train_processed:", np.isnan(X_train_processed).sum())
print("NaN в X_test_processed:", np.isnan(X_test_processed).sum())

NaN в X_train_processed: 0
NaN в X_test_processed: 0


Перейдём к модели

In [6]:
start_time = time.time()
knn = KNeighborsClassifier()
knn.fit(X_train_processed, y_train)
y_pred = knn.predict(X_test_processed)
results["KNN"] = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred, average='weighted')
}
print(f"Завершено за {time.time() - start_time:.2f} сек. F1: {results['KNN']['F1 Score']:.4f}")

Завершено за 534.32 сек. F1: 0.1455


Подберём оптимальные параметры, как мы это можем сделать, при условии, что модель обучается долго и у нас ограниченные ресурсы в первую очередь можно свести сетку параметров к миниму, во вторую настроить дропы используем RandomizedSearchCV, потому что он быстрее и его можно настроить.
Давайте отдельно расморим параметры которые мы выбираем, и параметры настройки RandomizedSearchCV и снизить параметры крос валидации потому что мы знаем что данные сбалансированы и нет выбросов

1)
weights - функция откоторой зависит с каким весом мы возьмём ближайшего соседа и т. д.

n_neighbors-количество соседей которых мы расматриваем при переборе

p-функция по которой мы считаем растояние до ближайшего соседа

2) n_iter количество случайных комбинаций парамеров для перебор

cv- насколько частей мы делим данные при крос валидации

n_jobs-упраляет ядрами процесора

refit после нахождения модели решет что мы с ней делаем(значени true-Автоматически обучает лучшую модель на ВСЕХ тренировочных данных)
    
verbose-управляет подробностью вывода на консоль

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import time

param_dist = {
    'n_neighbors': np.arange(3, 51, 2),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

knn_search = RandomizedSearchCV(
    KNeighborsClassifier(),
    param_dist,
    n_iter=10,
    cv=2,
    n_jobs=-1,
    refit=True,
    verbose=1,
    random_state=42
)


start_time = time.time()
knn_search.fit(X_train_processed, y_train)
search_time = time.time() - start_time

best_knn = knn_search.best_estimator_
y_pred = best_knn.predict(X_test_processed)

results = {
    'best_params': knn_search.best_params_,
    'time_sec': search_time,
    'test_metrics': {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1': f1_score(y_test, y_pred, average='weighted')
    }
}

print(f"Поиск завершен за {search_time:.2f} сек")
print(f"Лучшие параметры: {results['best_params']}")
print("Тестовые метрики:")
for metric, value in results['test_metrics'].items():
    print(f"{metric}: {value:.4f}")

Fitting 2 folds for each of 10 candidates, totalling 20 fits
